In [73]:
#cargar datos
import pandas as pd

datatest = pd.read_csv('C:\\Users\\danie\\Documents\\Analítica Predictiva\\2024-2-LAB-05-prediccion-de-precios-usando-linreg-dmolinav\\files\\input\\test_data.csv.zip')  
datatrain = pd.read_csv('C:\\Users\\danie\\Documents\\Analítica Predictiva\\2024-2-LAB-05-prediccion-de-precios-usando-linreg-dmolinav\\files\\input\\train_data.csv.zip')

#crear columna Age
datatrain['Age'] = 2021 - datatrain['Year']
datatest['Age'] = 2021 - datatest['Year']

#remover columna
datatrain.drop(['Year', 'Car_Name'], axis=1, inplace=True)
datatest.drop(['Year', 'Car_Name'], axis=1, inplace=True)



In [74]:
#dividir dataset
x_train = datatrain.drop(columns="Present_Price")
y_train = datatrain["Present_Price"]

x_test = datatest.drop(columns="Present_Price")
y_test = datatest["Present_Price"]

In [75]:
#Crear pipeline y ajustar modelo
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

oneHotEncoder = OneHotEncoder()
scaler = MinMaxScaler()

from sklearn.compose import ColumnTransformer

categorical_columns = ['Fuel_Type', 'Selling_type', 'Transmission']
numerical_columns = ['Selling_Price', 'Driven_kms', 'Owner', 'Age']

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_columns),
        ("scaler", MinMaxScaler(), numerical_columns)
    ],
)


selectKBest = SelectKBest(f_regression)

lr = LinearRegression()

estimators = [
    ("preprocessor", preprocessor), 
    ("kSelect", selectKBest),
    ("lr", lr),  
]

pipeline = Pipeline(
    steps=estimators,
    verbose=False,
)

In [76]:
#Validacion cruzada
from sklearn.model_selection import GridSearchCV
param_grid = {
    "kSelect__k": range(1,15),
    "lr__fit_intercept": [True, False],
    "lr__positive": [True, False]
}

gridSearchCV=GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

gridSearchCV.fit(x_train, y_train)

model=gridSearchCV

In [77]:
#guardar modelo
import os
import gzip
import pickle

model_filename = '../files/models/model.pkl.gz'
os.makedirs(os.path.dirname(model_filename), exist_ok=True) 

with gzip.open(model_filename, 'wb') as f:
    pickle.dump(model, f)

In [78]:
#metricas
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
import json

def calculate_metrics(model, X, y, dataset_name):
    y_pred = model.predict(X)
    
    r2_train = float(r2_score(y, y_pred))
    mse_train = float(mean_squared_error(y, y_pred))
    mad_train = float(median_absolute_error(y, y_pred))
    
    metrics = {
        'type': 'metrics',
        'dataset': dataset_name,
        'r2': r2_train,
        'mse': mse_train,
        'mad': mad_train,
    }
    return metrics

# Calcular métricas para el conjunto de entrenamiento y prueba
train_metrics = calculate_metrics(model, x_train, y_train, 'train')
test_metrics = calculate_metrics(model, x_test, y_test, 'test')

metrics = [train_metrics, test_metrics]

In [79]:
import json
import os


os.makedirs('../files/output', exist_ok=True)

def save_metrics(metrics, filename='../files/output/metrics.json'):
    with open(filename, 'a') as f:
        for metric in metrics:
            json.dump(metric, f)
            f.write('\n')

# Guardar las métricas calculadas
save_metrics(metrics)
